In [ ]:
!pip install dotenv

In [ ]:
import requests
import sqlite3
import os
from dotenv import load_dotenv
import sqlite3
import time




In [ ]:
# Load environment variables
load_dotenv()

# Get API keys from environment variable and split into a list
api_keys = os.getenv('API_KEY_ALPHAVANTAGE_sbabel_umass_edu', '').split(',')
if not api_keys or api_keys[0] == '':
    raise ValueError("No API keys found in the environment variable 'API_KEYS'.")


# 1. Read tickers from a .txt file

with open('../data/top60tickers.txt', 'r') as f:
    content = f.read().strip()
    # Remove any surrounding spaces and then split by comma
    tickers = [ticker.strip().strip("'").strip('"') for ticker in content.split(',') if ticker.strip()]


# Function to get the current API key based on index
current_key_index = 0
def get_current_api_key():
    return api_keys[current_key_index].strip()

# Function to rotate to the next API key
def rotate_api_key():
    global current_key_index
    current_key_index = (current_key_index + 1) % len(api_keys)
    print(f"Rotated to new API key: {get_current_api_key()}")

In [ ]:
# 2. Connect to SQLite database
conn = sqlite3.connect('../data/dbs/company_overview.db')
cursor = conn.cursor()

In [ ]:
# 3. Create table to store company overview data
cursor.execute('''
    CREATE TABLE IF NOT EXISTS company_overview (
        Symbol TEXT PRIMARY KEY,
        AssetType TEXT,
        Name TEXT,
        Description TEXT,
        CIK TEXT,
        Exchange TEXT,
        Currency TEXT,
        Country TEXT,
        Sector TEXT,
        Industry TEXT,
        Address TEXT,
        OfficialSite TEXT,
        FiscalYearEnd TEXT,
        LatestQuarter TEXT,
        MarketCapitalization TEXT,
        EBITDA TEXT,
        PERatio TEXT,
        PEGRatio TEXT,
        BookValue TEXT,
        DividendPerShare TEXT,
        DividendYield TEXT,
        EPS TEXT,
        RevenuePerShareTTM TEXT,
        ProfitMargin TEXT,
        OperatingMarginTTM TEXT,
        ReturnOnAssetsTTM TEXT,
        ReturnOnEquityTTM TEXT,
        RevenueTTM TEXT,
        GrossProfitTTM TEXT,
        DilutedEPSTTM TEXT,
        QuarterlyEarningsGrowthYOY TEXT,
        QuarterlyRevenueGrowthYOY TEXT,
        AnalystTargetPrice TEXT,
        AnalystRatingStrongBuy TEXT,
        AnalystRatingBuy TEXT,
        AnalystRatingHold TEXT,
        AnalystRatingSell TEXT,
        AnalystRatingStrongSell TEXT,
        TrailingPE TEXT,
        ForwardPE TEXT,
        PriceToSalesRatioTTM TEXT,
        PriceToBookRatio TEXT,
        EVToRevenue TEXT,
        EVToEBITDA TEXT,
        Beta TEXT,
        Week52High TEXT,
        Week52Low TEXT,
        MovingAverage50Day TEXT,
        MovingAverage200Day TEXT,
        SharesOutstanding TEXT,
        DividendDate TEXT,
        ExDividendDate TEXT
    )
''')
conn.commit()

In [ ]:
overview_arr = []

In [ ]:

# Loop through each ticker and fetch data using the current API key
for ticker in tickers:
    print(f"Fetching data for {ticker} using API key {get_current_api_key()}...")
    url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={ticker}&apikey={get_current_api_key()}'
    
    # Try to fetch data; if a limit error occurs, rotate to the next API key and retry
    success = False
    attempts = 0
    max_attempts = len(api_keys)  # Try each key once at most
    while not success and attempts < max_attempts:
        r = requests.get(url)
        data = r.json()
        
        # Check for expected key or for a note/error message indicating a rate limit
        if data and 'Symbol' in data:
            success = True
        elif data.get('Note') or data.get('Error Message'):
            print(f"API limit reached or error encountered for ticker {ticker} with key {get_current_api_key()}.")
            rotate_api_key()  # Switch API key
            url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={ticker}&apikey={get_current_api_key()}'
            attempts += 1
        else:
            print(f"No valid data for {ticker} using key {get_current_api_key()}.")
            rotate_api_key()
            url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={ticker}&apikey={get_current_api_key()}'
            attempts += 1
    
    if not success:
        print(f"Skipping {ticker} after trying all API keys.")
        continue

    # Prepare the overview data (defaulting missing fields to empty strings)
    overview = {
        'Symbol': data.get("Symbol", ""),
        'AssetType': data.get("AssetType", ""),
        'Name': data.get("Name", ""),
        'Description': data.get("Description", ""),
        'CIK': data.get("CIK", ""),
        'Exchange': data.get("Exchange", ""),
        'Currency': data.get("Currency", ""),
        'Country': data.get("Country", ""),
        'Sector': data.get("Sector", ""),
        'Industry': data.get("Industry", ""),
        'Address': data.get("Address", ""),
        'OfficialSite': data.get("OfficialSite", ""),
        'FiscalYearEnd': data.get("FiscalYearEnd", ""),
        'LatestQuarter': data.get("LatestQuarter", ""),
        'MarketCapitalization': data.get("MarketCapitalization", ""),
        'EBITDA': data.get("EBITDA", ""),
        'PERatio': data.get("PERatio", ""),
        'PEGRatio': data.get("PEGRatio", ""),
        'BookValue': data.get("BookValue", ""),
        'DividendPerShare': data.get("DividendPerShare", ""),
        'DividendYield': data.get("DividendYield", ""),
        'EPS': data.get("EPS", ""),
        'RevenuePerShareTTM': data.get("RevenuePerShareTTM", ""),
        'ProfitMargin': data.get("ProfitMargin", ""),
        'OperatingMarginTTM': data.get("OperatingMarginTTM", ""),
        'ReturnOnAssetsTTM': data.get("ReturnOnAssetsTTM", ""),
        'ReturnOnEquityTTM': data.get("ReturnOnEquityTTM", ""),
        'RevenueTTM': data.get("RevenueTTM", ""),
        'GrossProfitTTM': data.get("GrossProfitTTM", ""),
        'DilutedEPSTTM': data.get("DilutedEPSTTM", ""),
        'QuarterlyEarningsGrowthYOY': data.get("QuarterlyEarningsGrowthYOY", ""),
        'QuarterlyRevenueGrowthYOY': data.get("QuarterlyRevenueGrowthYOY", ""),
        'AnalystTargetPrice': data.get("AnalystTargetPrice", ""),
        'AnalystRatingStrongBuy': data.get("AnalystRatingStrongBuy", ""),
        'AnalystRatingBuy': data.get("AnalystRatingBuy", ""),
        'AnalystRatingHold': data.get("AnalystRatingHold", ""),
        'AnalystRatingSell': data.get("AnalystRatingSell", ""),
        'AnalystRatingStrongSell': data.get("AnalystRatingStrongSell", ""),
        'TrailingPE': data.get("TrailingPE", ""),
        'ForwardPE': data.get("ForwardPE", ""),
        'PriceToSalesRatioTTM': data.get("PriceToSalesRatioTTM", ""),
        'PriceToBookRatio': data.get("PriceToBookRatio", ""),
        'EVToRevenue': data.get("EVToRevenue", ""),
        'EVToEBITDA': data.get("EVToEBITDA", ""),
        'Beta': data.get("Beta", ""),
        'Week52High': data.get("52WeekHigh", ""),
        'Week52Low': data.get("52WeekLow", ""),
        'MovingAverage50Day': data.get("50DayMovingAverage", ""),
        'MovingAverage200Day': data.get("200DayMovingAverage", ""),
        'SharesOutstanding': data.get("SharesOutstanding", ""),
        'DividendDate': data.get("DividendDate", ""),
        'ExDividendDate': data.get("ExDividendDate", "")
    }

    overview_arr.append(overview)


In [ ]:
len(overview_arr)

In [ ]:
import requests
import sqlite3
import os
import time
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get API keys from environment variable and split into a list
api_keys = os.getenv('API_KEY_ALPHAVANTAGE_sbabel_umass_edu', '').split(',')
if not api_keys or api_keys[0] == '':
    raise ValueError("No API keys found in the environment variable 'API_KEYS'.")

# Function to get the current API key based on index
current_key_index = 0
def get_current_api_key():
    return api_keys[current_key_index].strip()

# Function to rotate to the next API key
def rotate_api_key():
    global current_key_index
    current_key_index = (current_key_index + 1) % len(api_keys)
    print(f"Rotated to new API key: {get_current_api_key()}")

# Read tickers from the text file (a single line with comma-separated tickers)
with open('../data/top60tickers.txt', 'r') as f:
    content = f.read().strip()
    tickers = [ticker.strip().strip("'").strip('"') for ticker in content.split(',') if ticker.strip()]

# Connect to SQLite database and create a table for company overviews
conn = sqlite3.connect('company_overview.db')
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS company_overview (
        Symbol TEXT PRIMARY KEY,
        AssetType TEXT,
        Name TEXT,
        Description TEXT,
        CIK TEXT,
        Exchange TEXT,
        Currency TEXT,
        Country TEXT,
        Sector TEXT,
        Industry TEXT,
        Address TEXT,
        OfficialSite TEXT,
        FiscalYearEnd TEXT,
        LatestQuarter TEXT,
        MarketCapitalization TEXT,
        EBITDA TEXT,
        PERatio TEXT,
        PEGRatio TEXT,
        BookValue TEXT,
        DividendPerShare TEXT,
        DividendYield TEXT,
        EPS TEXT,
        RevenuePerShareTTM TEXT,
        ProfitMargin TEXT,
        OperatingMarginTTM TEXT,
        ReturnOnAssetsTTM TEXT,
        ReturnOnEquityTTM TEXT,
        RevenueTTM TEXT,
        GrossProfitTTM TEXT,
        DilutedEPSTTM TEXT,
        QuarterlyEarningsGrowthYOY TEXT,
        QuarterlyRevenueGrowthYOY TEXT,
        AnalystTargetPrice TEXT,
        AnalystRatingStrongBuy TEXT,
        AnalystRatingBuy TEXT,
        AnalystRatingHold TEXT,
        AnalystRatingSell TEXT,
        AnalystRatingStrongSell TEXT,
        TrailingPE TEXT,
        ForwardPE TEXT,overview_arr
        PriceToSalesRatioTTM TEXT,
        PriceToBookRatio TEXT,
        EVToRevenue TEXT,
        EVToEBITDA TEXT,
        Beta TEXT,
        Week52High TEXT,
        Week52Low TEXT,
        MovingAverage50Day TEXT,
        MovingAverage200Day TEXT,
        SharesOutstanding TEXT,
        DividendDate TEXT,
        ExDividendDate TEXT
    )
''')
conn.commit()

# Loop through each ticker and fetch data using the current API key
for ticker in tickers:
    print(f"Fetching data for {ticker} using API key {get_current_api_key()}...")
    url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={ticker}&apikey={get_current_api_key()}'
    
    # Try to fetch data; if a limit error occurs, rotate to the next API key and retry
    success = False
    attempts = 0
    max_attempts = len(api_keys)  # Try each key once at most
    while not success and attempts < max_attempts:
        r = requests.get(url)
        data = r.json()
        
        # Check for expected key or for a note/error message indicating a rate limit
        if data and 'Symbol' in data:
            success = True
        elif data.get('Note') or data.get('Error Message'):
            print(f"API limit reached or error encountered for ticker {ticker} with key {get_current_api_key()}.")
            rotate_api_key()  # Switch API key
            url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={ticker}&apikey={get_current_api_key()}'
            attempts += 1
            # Optionally wait a bit before retrying (e.g., to respect rate limits)
            time.sleep(12)
        else:
            print(f"No valid data for {ticker} using key {get_current_api_key()}.")
            rotate_api_key()
            url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={ticker}&apikey={get_current_api_key()}'
            attempts += 1
            time.sleep(12)
    
    if not success:
        print(f"Skipping {ticker} after trying all API keys.")
        continue

    # Prepare the overview data (defaulting missing fields to empty strings)
    overview = {
        'Symbol': data.get("Symbol", ""),
        'AssetType': data.get("AssetType", ""),
        'Name': data.get("Name", ""),
        'Description': data.get("Description", ""),
        'CIK': data.get("CIK", ""),
        'Exchange': data.get("Exchange", ""),
        'Currency': data.get("Currency", ""),
        'Country': data.get("Country", ""),
        'Sector': data.get("Sector", ""),
        'Industry': data.get("Industry", ""),
        'Address': data.get("Address", ""),
        'OfficialSite': data.get("OfficialSite", ""),
        'FiscalYearEnd': data.get("FiscalYearEnd", ""),
        'LatestQuarter': data.get("LatestQuarter", ""),
        'MarketCapitalization': data.get("MarketCapitalization", ""),
        'EBITDA': data.get("EBITDA", ""),
        'PERatio': data.get("PERatio", ""),
        'PEGRatio': data.get("PEGRatio", ""),
        'BookValue': data.get("BookValue", ""),
        'DividendPerShare': data.get("DividendPerShare", ""),
        'DividendYield': data.get("DividendYield", ""),
        'EPS': data.get("EPS", ""),
        'RevenuePerShareTTM': data.get("RevenuePerShareTTM", ""),
        'ProfitMargin': data.get("ProfitMargin", ""),
        'OperatingMarginTTM': data.get("OperatingMarginTTM", ""),
        'ReturnOnAssetsTTM': data.get("ReturnOnAssetsTTM", ""),
        'ReturnOnEquityTTM': data.get("ReturnOnEquityTTM", ""),
        'RevenueTTM': data.get("RevenueTTM", ""),
        'GrossProfitTTM': data.get("GrossProfitTTM", ""),
        'DilutedEPSTTM': data.get("DilutedEPSTTM", ""),
        'QuarterlyEarningsGrowthYOY': data.get("QuarterlyEarningsGrowthYOY", ""),
        'QuarterlyRevenueGrowthYOY': data.get("QuarterlyRevenueGrowthYOY", ""),
        'AnalystTargetPrice': data.get("AnalystTargetPrice", ""),
        'AnalystRatingStrongBuy': data.get("AnalystRatingStrongBuy", ""),
        'AnalystRatingBuy': data.get("AnalystRatingBuy", ""),
        'AnalystRatingHold': data.get("AnalystRatingHold", ""),
        'AnalystRatingSell': data.get("AnalystRatingSell", ""),
        'AnalystRatingStrongSell': data.get("AnalystRatingStrongSell", ""),
        'TrailingPE': data.get("TrailingPE", ""),
        'ForwardPE': data.get("ForwardPE", ""),
        'PriceToSalesRatioTTM': data.get("PriceToSalesRatioTTM", ""),
        'PriceToBookRatio': data.get("PriceToBookRatio", ""),
        'EVToRevenue': data.get("EVToRevenue", ""),
        'EVToEBITDA': data.get("EVToEBITDA", ""),
        'Beta': data.get("Beta", ""),
        'Week52High': data.get("52WeekHigh", ""),
        'Week52Low': data.get("52WeekLow", ""),
        'MovingAverage50Day': data.get("50DayMovingAverage", ""),
        'MovingAverage200Day': data.get("200DayMovingAverage", ""),
        'SharesOutstanding': data.get("SharesOutstanding", ""),
        'DividendDate': data.get("DividendDate", ""),
        'ExDividendDate': data.get("ExDividendDate", "")
    }
    overview_arr.append(overview)
    # # Insert the data into the SQLite database
    # cursor.execute('''
    #     INSERT OR REPLACE INTO company_overview (
    #         Symbol, AssetType, Name, Description, CIK, Exchange, Currency, Country, Sector,
    #         Industry, Address, OfficialSite, FiscalYearEnd, LatestQuarter, MarketCapitalization,
    #         EBITDA, PERatio, PEGRatio, BookValue, DividendPerShare, DividendYield, EPS,
    #         RevenuePerShareTTM, ProfitMargin, OperatingMarginTTM, ReturnOnAssetsTTM, ReturnOnEquityTTM,
    #         RevenueTTM, GrossProfitTTM, DilutedEPSTTM, QuarterlyEarningsGrowthYOY, QuarterlyRevenueGrowthYOY,
    #         AnalystTargetPrice, AnalystRatingStrongBuy, AnalystRatingBuy, AnalystRatingHold,
    #         AnalystRatingSell, AnalystRatingStrongSell, TrailingPE, ForwardPE, PriceToSalesRatioTTM,import subprocess

def connect_nordvpn(country='United_States'):
    # Connect to a server in a specific country
    result = subprocess.run(["nordvpn", "connect", country], capture_output=True, text=True)
    print(result.stdout)
    if result.returncode != 0:
        print("Error connecting:", result.stderr)

def disconnect_nordvpn():
    result = subprocess.run(["nordvpn", "disconnect"], capture_output=True, text=True)
    print(result.stdout)
    if result.returncode != 0:
        print("Error disconnecting:", result.stderr)

# # Example usage
# connect_nordvpn('United_States')
# # Do your proxy requests here...
# disconnect_nordvpn()

    #         PriceToBookRatio, EVToRevenue, EVToEBITDA, Beta, Week52High, Week52Low,
    #         MovingAverage50Day, MovingAverage200Day, SharesOutstanding, DividendDate, ExDividendDate
    #     ) VALUES (
    #         :Symbol, :AssetType, :Name, :Description, :CIK, :Exchange, :Currency, :Country, :Sector,
    #         :Industry, :Address, :OfficialSite, :FiscalYearEnd, :LatestQuarter, :MarketCapitalization,
    #         :EBITDA, :PERatio, :PEGRatio, :BookValue, :DividendPerShare, :DividendYield, :EPS,
    #         :RevenuePerShareTTM, :ProfitMargin, :OperatingMarginTTM, :ReturnOnAssetsTTM, :ReturnOnEquityTTM,
    #         :RevenueTTM, :GrossProfitTTM, :DilutedEPSTTM, :QuarterlyEarningsGrowthYOY, :QuarterlyRevenueGrowthYOY,
    #         :AnalystTargetPrice, :AnalystRatingStrongBuy, :AnalystRatingBuy, :AnalystRatingHold,
    #         :AnalystRatingSell, :AnalystRatingStrongSell, :TrailingPE, :ForwardPE, :PriceToSalesRatioTTM,
    #         :PriceToBookRatio, :EVToRevenue, :EVToEBITDA, :Beta, :Week52High, :Week52Low,
    #         :MovingAverage50Day, :MovingAverage200Day, :SharesOutstanding, :DividendDate, :ExDividendDate
    #     )
    # ''', overview)
    
    # conn.commit()
    # Optionally wait a bit between ticker requests to avoid rapid-fire requests
    time.sleep(12)

# Close the database connection
conn.close()
print("Data for all tickers has been saved to the database.")


In [ ]:

# Insert the data into the SQLite database
cursor.execute('''
    INSERT OR REPLACE INTO company_overview (
        Symbol, AssetType, Name, Description, CIK, Exchange, Currency, Country, Sector,
        Industry, Address, OfficialSite, FiscalYearEnd, LatestQuarter, MarketCapitalization,
        EBITDA, PERatio, PEGRatio, BookValue, DividendPerShare, DividendYield, EPS,
        RevenuePerShareTTM, ProfitMargin, OperatingMarginTTM, ReturnOnAssetsTTM, ReturnOnEquityTTM,
        RevenueTTM, GrossProfitTTM, DilutedEPSTTM, QuarterlyEarningsGrowthYOY, QuarterlyRevenueGrowthYOY,
        AnalystTargetPrice, AnalystRatingStrongBuy, AnalystRatingBuy, AnalystRatingHold,
        AnalystRatingSell, AnalystRatingStrongSell, TrailingPE, ForwardPE, PriceToSalesRatioTTM,
        PriceToBookRatio, EVToRevenue, EVToEBITDA, Beta, Week52High, Week52Low,
        MovingAverage50Day, MovingAverage200Day, SharesOutstanding, DividendDate, ExDividendDate
    ) VALUES (
        :Symbol, :AssetType, :Name, :Description, :CIK, :Exchange, :Currency, :Country, :Sector,
        :Industry, :Address, :OfficialSite, :FiscalYearEnd, :LatestQuarter, :MarketCapitalization,
        :EBITDA, :PERatio, :PEGRatio, :BookValue, :DividendPerShare, :DividendYield, :EPS,
        :RevenuePerShareTTM, :ProfitMargin, :OperatingMarginTTM, :ReturnOnAssetsTTM, :ReturnOnEquityTTM,
        :RevenueTTM, :GrossProfitTTM, :DilutedEPSTTM, :QuarterlyEarningsGrowthYOY, :QuarterlyRevenueGrowthYOY,
        :AnalystTargetPrice, :AnalystRatingStrongBuy, :AnalystRatingBuy, :AnalystRatingHold,
        :AnalystRatingSell, :AnalystRatingStrongSell, :TrailingPE, :ForwardPE, :PriceToSalesRatioTTM,
        :PriceToBookRatio, :EVToRevenue, :EVToEBITDA, :Beta, :Week52High, :Week52Low,
        :MovingAverage50Day, :MovingAverage200Day, :SharesOutstanding, :DividendDate, :ExDividendDate
    )
''', overview)

conn.commit()

# Close the database connection
conn.close()
print("Data for all tickers has been saved to the database.")


In [ ]:
import requests
url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol=PLTR&apikey=NRWNE3ZL80JWH069'
    
r = requests.get(url)
data = r.json()
data

In [ ]:
from fetch_funcs import fetch_intraday_data
ticker = 'PLTR'
year = 2016
month = 1
api_key = 'NRWNE3ZL80JWH069'
fetch_intraday_data(ticker, api_key, year=year, month=month)

In [ ]:
data

In [ ]:
from fetch_funcs import fetch_company_overview
from helpers import connect_nordvpn, disconnect_nordvpn
import requests
import sqlite3
import os
import time
from dotenv import load_dotenv

In [ ]:
# Load environment variables from .env file
load_dotenv()

# Get API keys from environment variable and split into a list
api_keys = os.getenv('API_KEY_ALPHAVANTAGE_sbabel_umass_edu', '').split(',')
if not api_keys or api_keys[0] == '':
    raise ValueError("No API keys found in the environment variable 'API_KEYS'.")

# Function to get the current API key based on index
current_key_index = 0
def get_current_api_key():
    return api_keys[current_key_index].strip()

# Function to rotate to the next API key
def rotate_api_key():
    global current_key_index
    current_key_index = (current_key_index + 1) % len(api_keys)
    print(f"Rotated to new API key: {get_current_api_key()}")

# Read tickers from the text file (a single line with comma-separated tickers)
with open('../data/top60tickers.txt', 'r') as f:
    content = f.read().strip()
    tickers = [ticker.strip().strip("'").strip('"') for ticker in content.split(',') if ticker.strip()]

# Connect to SQLite database and create a table for company overviews
conn = sqlite3.connect('company_overview.db')
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS company_overview (
        Symbol TEXT PRIMARY KEY,
        AssetType TEXT,
        Name TEXT,
        Description TEXT,
        CIK TEXT,
        Exchange TEXT,
        Currency TEXT,
        Country TEXT,
        Sector TEXT,
        Industry TEXT,
        Address TEXT,
        OfficialSite TEXT,
        FiscalYearEnd TEXT,
        LatestQuarter TEXT,
        MarketCapitalization TEXT,
        EBITDA TEXT,
        PERatio TEXT,
        PEGRatio TEXT,
        BookValue TEXT,
        DividendPerShare TEXT,
        DividendYield TEXT,
        EPS TEXT,
        RevenuePerShareTTM TEXT,
        ProfitMargin TEXT,
        OperatingMarginTTM TEXT,
        ReturnOnAssetsTTM TEXT,
        ReturnOnEquityTTM TEXT,
        RevenueTTM TEXT,
        GrossProfitTTM TEXT,
        DilutedEPSTTM TEXT,
        QuarterlyEarningsGrowthYOY TEXT,
        QuarterlyRevenueGrowthYOY TEXT,
        AnalystTargetPrice TEXT,
        AnalystRatingStrongBuy TEXT,
        AnalystRatingBuy TEXT,
        AnalystRatingHold TEXT,
        AnalystRatingSell TEXT,
        AnalystRatingStrongSell TEXT,
        TrailingPE TEXT,
        ForwardPE TEXT,overview_arr
        PriceToSalesRatioTTM TEXT,
        PriceToBookRatio TEXT,
        EVToRevenue TEXT,
        EVToEBITDA TEXT,
        Beta TEXT,
        Week52High TEXT,
        Week52Low TEXT,
        MovingAverage50Day TEXT,
        MovingAverage200Day TEXT,
        SharesOutstanding TEXT,
        DividendDate TEXT,
        ExDividendDate TEXT
    )
''')
conn.commit()


In [ ]:
overview_arr = []

for ticker in tickers:
    success = False
    attempts = 0
    max_attempts = len(api_keys)

    while not success and attempts < max_attempts:
        current_key = get_current_api_key()
        overview = fetch_company_overview(ticker, current_key)

        # Check if valid data was returned
        if overview != 'no data':
            overview_arr.append(overview)
            success = True
        else:
            print(f"Failed to fetch overview for {ticker} with API key {current_key}")
            rotate_api_key()
            attempts += 1
            disconnect_nordvpn()
            connect_nordvpn()
            # time.sleep(12)  # Respect API rate limit

    if not success:
        print(f"Skipping {ticker} after trying all API keys.")


In [ ]:
import os

# print("PATH:", os.environ["PATH"])


In [ ]:
disconnect_nordvpn()

In [ ]:
!nordvpn

In [ ]:
os.system('which nordvpn')

In [ ]:
!python3 helpers.py

In [ ]:
import sqlite3

conn = sqlite3.connect('company_overview.db')
cursor = conn.cursor()
cursor.execute("PRAGMA table_info(company_overview);")
columns = cursor.fetchall()
for column in columns:
    print(column)
# conn.close()


In [ ]:

cursor.execute('''drop table company_overview''')

In [ ]:
conn = sqlite3.connect('finance_data.db')
conn.execute('SELECT distinct Symbol FROM company_intraday_data')
rows = list(conn.fetchall())  # Fetch all results
for row in rows:
    print(row[0])
print(rows)


In [ ]:
import sqlite3

conn = sqlite3.connect('finance_data.db')
cursor = conn.cursor()
cursor.execute("select * from company_intraday_data")
columns = cursor.fetchall()
for column in columns:
    print(column)
# conn.close()

In [ ]:
from fetch_funcs import fetch_intraday_data
import sqlite3

fetch_intraday_data('AAPL', '', interval='60min', year=2016, month=1)

In [ ]:
year = 2016
month = 1
ticker = 'AAPL'

conn = sqlite3.connect('finance_data.db')
cursor = conn.cursor()

exe_str = """"""

cursor.execute('''
SELECT COUNT(*) FROM company_intraday_data
WHERE symbol = ? AND strftime('%Y-%m', datetime) = ?
''', (ticker, f"{year}-{month:02d}"))
exists = cursor.fetchone()[0] > 0

conn.close()

if exists:
    print(f"Data for {ticker} for {year}-{month:02d} already exists. Skipping fetch.")

In [ ]:
import re
txt = 'Welcome to Alpha Vantage! Your API key is: 7N5A1LMRZEHPR2KR. Please record this API key at a safe place for future data access.'

match = re.search(r'API key\s*(?:is:|:)\s*([A-Z0-9]+)', txt)
print(match)

In [ ]:
import re

txt1 = 'Your API key is: 7N5A1LMRZEHPR2KR'
txt2 = 'Your API key: 7N5A1LMRZEHPR2KR'

pattern = r'key\s*(?:is:|:)\s*([A-Z0-9]+)'

for txt in [txt1, txt2]:
    match = re.search(pattern, txt)
    if match:
        print("Matched key:", match.group(1))
    else:
        print("No match found.")


In [ ]:
import sqlite3
import csv

def export_to_csv(db_file, table_name, csv_file):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    
    # Fetch all data from the table
    cursor.execute(f"SELECT * FROM {table_name}")
    rows = cursor.fetchall()
    
    # Get column names
    column_names = [desc[0] for desc in cursor.description]

    # Write to CSV
    with open(csv_file, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(column_names)  # Write headers
        writer.writerows(rows)  # Write data

    # Close connection
    conn.close()
    print(f"Exported {table_name} to {csv_file}")




def export_all_tables(db_file):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Get all table names
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [table[0] for table in cursor.fetchall()]

    for table in tables:
        print(table)
        export_to_csv(db_file, table, f"data/csv/{table}.csv")

    conn.close()

# Example usage
export_all_tables("finance_data.db")


In [ ]:
conn = sqlite3.connect('finance_data.db')
cursor = conn.cursor()


# cursor.execute('''
#     drop table income_statement
#     ''')

cursor.execute('''
        CREATE TABLE IF NOT EXISTS income_statement (
            symbol TEXT,
            fiscalDateEnding TEXT,
            reportType TEXT,
            reportedCurrency TEXT,
            grossProfit TEXT,
            totalRevenue TEXT,
            costOfRevenue TEXT,
            costofGoodsAndServicesSold TEXT,
            operatingIncome TEXT,
            sellingGeneralAndAdministrative TEXT,
            researchAndDevelopment TEXT,
            operatingExpenses TEXT,
            investmentIncomeNet TEXT,
            netInterestIncome TEXT,
            interestIncome TEXT,
            interestExpense TEXT,
            nonInterestIncome TEXT,
            otherNonOperatingIncome TEXT,
            depreciation TEXT,
            depreciationAndAmortization TEXT,
            incomeBeforeTax TEXT,
            incomeTaxExpense TEXT,
            interestAndDebtExpense TEXT,
            netIncomeFromContinuingOperations TEXT,
            comprehensiveIncomeNetOfTax TEXT,
            ebit TEXT,
            ebitda TEXT,
            netIncome TEXT,
            PRIMARY KEY (symbol, fiscalDateEnding, reportType)
        )
        ''')


In [ ]:
conn = sqlite3.connect('finance_data.db')
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS balance_sheet (
        symbol TEXT,
        fiscalDateEnding TEXT,
        reportType TEXT,
        reportedCurrency TEXT,
        totalAssets TEXT,
        totalCurrentAssets TEXT,
        cashAndCashEquivalentsAtCarryingValue TEXT,
        cashAndShortTermInvestments TEXT,
        inventory TEXT,
        currentNetReceivables TEXT,
        totalNonCurrentAssets TEXT,
        propertyPlantEquipment TEXT,
        accumulatedDepreciationAmortizationPPE TEXT,
        intangibleAssets TEXT,
        intangibleAssetsExcludingGoodwill TEXT,
        goodwill TEXT,
        investments TEXT,
        longTermInvestments TEXT,
        shortTermInvestments TEXT,
        otherCurrentAssets TEXT,
        otherNonCurrentAssets TEXT,
        totalLiabilities TEXT,
        totalCurrentLiabilities TEXT,
        currentAccountsPayable TEXT,
        deferredRevenue TEXT,
        currentDebt TEXT,
        shortTermDebt TEXT,
        totalNonCurrentLiabilities TEXT,
        capitalLeaseObligations TEXT,
        longTermDebt TEXT,
        currentLongTermDebt TEXT,
        longTermDebtNoncurrent TEXT,
        shortLongTermDebtTotal TEXT,
        otherCurrentLiabilities TEXT,
        otherNonCurrentLiabilities TEXT,
        totalShareholderEquity TEXT,
        treasuryStock TEXT,
        retainedEarnings TEXT,
        commonStock TEXT,
        commonStockSharesOutstanding TEXT,
        PRIMARY KEY (symbol, fiscalDateEnding, reportType)
    )
    ''')

In [ ]:
conn = sqlite3.connect('finance_data.db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS cash_flow (
    symbol TEXT,
    fiscalDateEnding TEXT,
    reportType TEXT,
    reportedCurrency TEXT,
    operatingCashflow TEXT,
    paymentsForOperatingActivities TEXT,
    proceedsFromOperatingActivities TEXT,
    changeInOperatingLiabilities TEXT,
    changeInOperatingAssets TEXT,
    depreciationDepletionAndAmortization TEXT,
    capitalExpenditures TEXT,
    changeInReceivables TEXT,
    changeInInventory TEXT,
    profitLoss TEXT,
    cashflowFromInvestment TEXT,
    cashflowFromFinancing TEXT,
    proceedsFromRepaymentsOfShortTermDebt TEXT,
    paymentsForRepurchaseOfCommonStock TEXT,
    paymentsForRepurchaseOfEquity TEXT,
    paymentsForRepurchaseOfPreferredStock TEXT,
    dividendPayout TEXT,
    dividendPayoutCommonStock TEXT,
    dividendPayoutPreferredStock TEXT,
    proceedsFromIssuanceOfCommonStock TEXT,
    proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet TEXT,
    proceedsFromIssuanceOfPreferredStock TEXT,
    proceedsFromRepurchaseOfEquity TEXT,
    proceedsFromSaleOfTreasuryStock TEXT,
    changeInCashAndCashEquivalents TEXT,
    changeInExchangeRate TEXT,
    netIncome TEXT,
    PRIMARY KEY (symbol, fiscalDateEnding, reportType)
)
''')

In [ ]:
# conn = sqlite3.connect('finance_data.db')
# cursor = conn.cursor()

# cursor.execute('''
# drop table balance_sheet
            
#     ''')

In [ ]:
import sqlite3
conn = sqlite3.connect('finance_data.db')
cursor = conn.cursor()

In [ ]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS news_articles (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT,
    url TEXT UNIQUE,
    time_published TEXT,
    authors TEXT,
    summary TEXT,
    banner_image TEXT,
    source TEXT,
    category_within_source TEXT,
    source_domain TEXT,
    topics TEXT,
    overall_sentiment_score REAL,
    overall_sentiment_label TEXT,
    fetch_date TEXT
)
''')

# Create news ticker mentions table (for the ticker-specific sentiment data)
cursor.execute('''
CREATE TABLE IF NOT EXISTS news_ticker_sentiment (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    article_id INTEGER,
    ticker_symbol TEXT,
    relevance_score REAL,
    ticker_sentiment_score REAL,
    ticker_sentiment_label TEXT,
    FOREIGN KEY (article_id) REFERENCES news_articles (id),
    UNIQUE (article_id, ticker_symbol)
)
''')

conn.commit()

In [ ]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
for table in tables:
    print(table[0])

In [ ]:
# Connect to both databases
company_db_conn = sqlite3.connect('company_overview.db')
finance_db_conn = sqlite3.connect('finance_data.db')

# Get a cursor for each database
company_cursor = company_db_conn.cursor()
finance_cursor = finance_db_conn.cursor()

# Read data from the company_overview table in company_overview.db
company_cursor.execute("SELECT * FROM company_overview")
rows = company_cursor.fetchall()

# Get the table structure (column names and types)
company_cursor.execute("PRAGMA table_info(company_overview);")
columns = company_cursor.fetchall()
column_definitions = ", ".join([f"{col[1]} {col[2]}" for col in columns])

# Create the table in finance_data.db if it doesn't exist
finance_cursor.execute(f"CREATE TABLE IF NOT EXISTS company_overview ({column_definitions})")

# Insert the data into the finance_data.db table
for row in rows:
    placeholders = ", ".join(["?" for _ in row])
    finance_cursor.execute(f"INSERT OR REPLACE INTO company_overview VALUES ({placeholders})", row)

# Commit changes and close connections
finance_db_conn.commit()
company_db_conn.close()
finance_db_conn.close()

print("Table successfully transferred from company_overview.db to finance_data.db.")

In [ ]:
import sqlite3
conn = sqlite3.connect('finance_data.db')
cursor = conn.cursor()

# Get table structure information
cursor.execute("PRAGMA table_info(news_articles)")
columns_info = cursor.fetchall()

# Display column information
print("Column Name - Type")
print("-----------------")
for col in columns_info:
    # col format: (cid, name, type, notnull, default_value, pk)
    col_id, name, data_type, not_null, default_val, primary_key = col
    print(f"{name} - {data_type}")

In [ ]:
# Query with ORDER BY on the published time column
cursor.execute("SELECT * FROM news_articles ORDER BY time_published DESC LIMIT 50")
# Or if the column has a different name, replace 'published_time' with the actual column name

ordered_articles = cursor.fetchall()
for article in ordered_articles[:5]:  # Show just first 5 results
    print(article)

In [ ]:
# Query with ORDER BY on the published time column
cursor.execute("SELECT DISTINCT ticker_symbol, count(ticker_symbol) FROM  news_ticker_sentiment group by ticker_symbol")
# Or if the column has a different name, replace 'published_time' with the actual column name

ordered_articles = cursor.fetchall()
for article in ordered_articles:  # Show just first 5 results
    print(article)

In [ ]:
import sqlite3
conn = sqlite3.connect(r'D:\Shared Data\College\Spring 2025\Final Project\CS571-Finance-Sentiment\app\data\finance_data.db')
cursor = conn.cursor()
cursor.execute("select * from news_articles limit 50")
columns = cursor.fetchall()
for column in columns:
    print(column)

In [ ]:
import sqlite3
conn = sqlite3.connect(r'D:\Shared Data\College\Spring 2025\Final Project\CS571-Finance-Sentiment\app\data\finance_data.db')
cursor = conn.cursor()
cursor.execute("select * from cash_flow limit 50")
columns = cursor.fetchall()
for column in columns:
    print(column)